In [26]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

   
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import json
import requests 

import pandas as pd
import requests
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [49]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

## First part of the assignment

In [69]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))

In [70]:
data = pd.DataFrame(df[0])

In [71]:
data_processed = data[data['Borough']!='Not assigned']

In [72]:
data_processed=data_processed.groupby('Postcode').agg(lambda x:','.join(set(x)))

In [73]:
data_processed.loc[data_processed['Neighbourhood']=='Not assigned','Neighbourhood'] = data_processed.loc[data_processed['Neighbourhood']=='Not assigned','Borough']

In [74]:
data_processed.shape

(103, 2)

### Final required output for the 1st part of assignment

In [ ]:
data_processed

## Second Part of the Assigment

In [75]:
!wget -q -O 'newyork_data.csv' http://cocl.us/Geospatial_data

In [76]:
post_code = pd.read_csv('newyork_data.csv')

In [77]:
data_processed = data_processed.merge(post_code,left_on='Postcode',right_on='Postal Code')

In [78]:
data_processed.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"West Hill,Guildwood,Morningside",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Third Part of the assignment

In [79]:
CLIENT_ID = 'VEC3ZDLCCA1DS2DXDTG45MHFCPN4ZKWIHUEAP25AK3CB0F4G'
CLIENT_SECRET = 'UMKXIMLLVWJIGKQTNK3SUUBWTBPHQZUQHZMQWSJL1XYK05DB'
VERSION = '20180605'
LIMIT = 30

In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
toronto_venues = getNearbyVenues(names=data_processed['Neighbourhood'],
                                   latitudes=data_processed['Latitude'],
                                   longitudes=data_processed['Longitude']
                                  )

Rouge,Malvern
Port Union,Rouge Hill,Highland Creek
West Hill,Guildwood,Morningside
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Oakridge,Golden Mile
Scarborough Village West,Cliffside,Cliffcrest
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Wexford,Maryvale
Agincourt
Sullivan,Tam O'Shanter,Clarks Corners
Milliken,L'Amoreaux East,Agincourt North,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Oriole,Henry Farm,Fairview
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Wilson Heights,Downsview North,Bathurst Manor
York University,Northwood Park
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale,The Danforth West
India Bazaar,The Beac

In [82]:
print(toronto_venues.shape)
toronto_venues.head()

(1331, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"West Hill,Guildwood,Morningside",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"West Hill,Guildwood,Morningside",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [83]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 242 uniques categories.


In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Port Union,Rouge Hill,Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"West Hill,Guildwood,Morningside",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"West Hill,Guildwood,Morningside",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring the surrounding areas

In [85]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
1,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,"Birch Cliff,Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
5,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.052632
6,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.2,0.000000
7,Canada Post Gateway Processing Centre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
8,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
9,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [86]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
               venue  freq
0             Lounge  0.25
1     Clothing Store  0.25
2       Skating Rink  0.25
3     Breakfast Spot  0.25
4  Accessories Store  0.00


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place   0.2
1                 Gym   0.1
2  Athletics & Sports   0.1
3        Skating Rink   0.1
4      Sandwich Place   0.1


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.07
2    Cocktail Bar  0.07
3        Beer Bar  0.07
4  Farmers Market  0.07


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4      Accessories Store  0.00


----Business Reply Mail Proce

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
1,"Alderwood,Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Gym,Antique Shop
2,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
3,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Café,Basketball Stadium,Bistro,Breakfast Spot,Clothing Store
4,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## Clustering the area into 5 clusters

In [89]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 0, 4, 3, 0, 4, 0], dtype=int32)

In [90]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [91]:

toronto_merged = data_processed

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353,4.0,Print Shop,Fast Food Restaurant,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",M1C,43.784535,-79.160497,2.0,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,Scarborough,"West Hill,Guildwood,Morningside",M1E,43.763573,-79.188711,4.0,Pizza Place,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Intersection,Breakfast Spot,Costume Shop,Discount Store,Event Space
3,Scarborough,Woburn,M1G,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,4.0,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Lounge,Curling Ice,Dance Studio


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
6,"East Birchmount Park,Ionview,Kennedy Park",0.0,Coffee Shop,Convenience Store,Department Store,Discount Store,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,"Birch Cliff,Cliffside West",0.0,College Stadium,Café,Skating Rink,General Entertainment,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
19,Bayview Village,0.0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
22,Willowdale South,0.0,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Restaurant,Ice Cream Shop,Sushi Restaurant,Indonesian Restaurant,Steakhouse,Fast Food Restaurant
24,Willowdale West,0.0,Pharmacy,Coffee Shop,Discount Store,Butcher,Pizza Place,Grocery Store,Construction & Landscaping,Concert Hall,Falafel Restaurant,Event Space
26,Don Mills North,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
27,"Don Mills South,Flemingdon Park",0.0,Gym,Beer Store,Coffee Shop,Asian Restaurant,Discount Store,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant,Bike Shop,Chinese Restaurant
29,"York University,Northwood Park",0.0,Massage Studio,Falafel Restaurant,Coffee Shop,Bar,Caribbean Restaurant,Diner,Farmers Market,Event Space,Ethiopian Restaurant,Empanada Restaurant
33,Downsview Northwest,0.0,Gym / Fitness Center,Grocery Store,Coffee Shop,Liquor Store,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Drugstore,Dumpling Restaurant


In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills,York Mills",1.0,Cafeteria,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Port Union,Rouge Hill,Highland Creek",2.0,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken,L'Amoreaux East,Agincourt North,Steel...",3.0,Playground,Gym,Park,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
23,York Mills West,3.0,Convenience Store,Park,Bank,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
25,Parkwoods,3.0,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
30,"Downsview East,CFB Toronto",3.0,Airport,Park,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
40,East Toronto,3.0,Park,Pizza Place,Coffee Shop,Convenience Store,Electronics Store,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant
44,Lawrence Park,3.0,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
50,Rosedale,3.0,Park,Building,Trail,Playground,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Yoga Studio
64,"Forest Hill West,Forest Hill North",3.0,Park,Jewelry Store,Sushi Restaurant,Trail,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Drugstore,Eastern European Restaurant
74,Caledonia-Fairbanks,3.0,Park,Women's Store,Fast Food Restaurant,Market,Convenience Store,Cosmetics Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
79,"North Park,Downsview,Upwood Park",3.0,Construction & Landscaping,Park,Bakery,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",4.0,Print Shop,Fast Food Restaurant,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
2,"West Hill,Guildwood,Morningside",4.0,Pizza Place,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Intersection,Breakfast Spot,Costume Shop,Discount Store,Event Space
4,Cedarbrae,4.0,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Athletics & Sports,Lounge,Curling Ice,Dance Studio
5,Scarborough Village,4.0,Playground,Health & Beauty Service,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
7,"Clairlea,Oakridge,Golden Mile",4.0,Bus Line,Fast Food Restaurant,Bus Station,Soccer Field,Park,Bakery,Intersection,Metro Station,Drugstore,Dumpling Restaurant
8,"Scarborough Village West,Cliffside,Cliffcrest",4.0,American Restaurant,Motel,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
10,"Dorset Park,Scarborough Town Centre,Wexford He...",4.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Furniture / Home Store,Cosmetics Shop,Diner,Falafel Restaurant,Event Space
11,"Wexford,Maryvale",4.0,Breakfast Spot,Smoke Shop,Middle Eastern Restaurant,Bakery,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
12,Agincourt,4.0,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
13,"Sullivan,Tam O'Shanter,Clarks Corners",4.0,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Bank,Italian Restaurant,Fried Chicken Joint,Thai Restaurant,Rental Car Location,Breakfast Spot
